In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import math
from matplotlib.backend_bases import MouseButton
matplotlib.use('TkAgg')

def distance(position, xprey, yprey):
    return np.sqrt((position[0] - xprey)**2 + (position[1] - yprey)**2)

plt.figure(figsize=(8, 8))
plt.grid(True)
plt.xlim(0, 100)
plt.ylim(0, 100)
plt.title('Click to place prey (first) and Aquilas (next clicks)\nRight click when done')

positions = []
prey_pos = None

def onclick(event):
    global prey_pos
    if event.button is MouseButton.RIGHT:
        plt.close()
        return

    if event.button is MouseButton.LEFT:
        if prey_pos is None:
            prey_pos = np.array([event.xdata, event.ydata])
            plt.plot(prey_pos[0], prey_pos[1], 'ks', markersize=12, markerfacecolor='magenta')
        else:
            positions.append([event.xdata, event.ydata])
            plt.plot(event.xdata, event.ydata, 'ro', markersize=8, markerfacecolor='blue')
        plt.draw()

plt.connect('button_press_event', onclick)
plt.show()

if len(positions) == 0 or prey_pos is None:
    print("No positions selected. Exiting.")
    exit()

positions = np.array(positions)
num_aquilas = len(positions)
max_iter = 500
dim = 2
lb = 0
ub = 100

# AO Parameters
alpha = 0.1
delta = 0.1
beta = 1.8  # For Lévy flight

X_best = positions[0].copy()
best_fitness = float('inf')
convergence_curve = np.zeros(max_iter)

# Initialize best solution
for i in range(num_aquilas):
    current_fitness = distance(positions[i], prey_pos[0], prey_pos[1])
    if current_fitness < best_fitness:
        best_fitness = current_fitness
        X_best = positions[i].copy()

def reflective_boundaries(position, lb, ub):
    position = np.where(position < lb, 2 * lb - position, position)
    position = np.where(position > ub, 2 * ub - position, position)
    return position

plt.figure(figsize=(8, 8))
for iter in range(max_iter):
    plt.clf()
    plt.grid(True)
    plt.xlim(-10, 110)
    plt.ylim(-10, 110)

    plt.plot(prey_pos[0], prey_pos[1], 'ks', markersize=12, markerfacecolor='magenta')
    plt.plot(positions[:, 0], positions[:, 1], 'ro', markersize=8, markerfacecolor='blue')
    plt.plot(X_best[0], X_best[1], 'g*', markersize=15, markerfacecolor='lime')

    a = 2 * (1 - iter/max_iter)  # Exploration-exploitation balance

    # Calculate weighted mean position
    fitness_values = np.array([distance(pos, prey_pos[0], prey_pos[1]) for pos in positions])
    weights = 1 / (fitness_values + np.finfo(float).eps)
    weighted_mean = np.sum(positions * weights[:, np.newaxis], axis=0) / np.sum(weights)

    for i in range(num_aquilas):
        if iter <= (2/3)*max_iter:  # Exploration phase
            if np.random.rand() < 0.5:
                # Expanded exploration (High soar with vertical stoop) with weighted mean
                positions[i] = X_best * (1 - iter/max_iter) + (weighted_mean - X_best) * np.random.rand()
            else:
                # Narrowed exploration (Contour flight with short glide) with weighted mean
                r = np.random.rand()
                theta = np.random.rand() * 2 * np.pi
                x = r * np.sin(theta)
                y = r * np.cos(theta)

                # Lévy flight calculation
                sigma = (np.math.gamma(1+beta)*np.sin(np.pi*beta/2)/(np.math.gamma((1+beta)/2)*beta*2**((beta-1)/2)))**(1/beta)
                u = np.random.randn(dim) * sigma
                v = np.random.randn(dim)
                levy = 0.01 * u / np.abs(v)**(1/beta)

                positions[i] = X_best * levy + positions[np.random.randint(num_aquilas)] + (y - x) * np.random.rand()
        else:  # Exploitation phase
            if np.random.rand() < 0.5:
                # Expanded exploitation (Low flight with gradual descent) with weighted mean
                positions[i] = (X_best * weighted_mean) * alpha - np.random.rand() + ((ub - lb) * np.random.rand() + lb) * delta
            else:
                # Narrowed exploitation (Walk and grab prey) with weighted mean
                QF = iter**((2*np.random.rand()-1)/(1 - max_iter)**2)
                G1 = 2 * np.random.rand() - 1
                G2 = 2 * (1 - iter/max_iter)
                levy = np.random.rand()  # Simplified Lévy for visualization

                positions[i] = QF * X_best - G1 * positions[i] * np.random.rand() - G2 * levy + np.random.rand()

        # Reflective Boundary Handling
        positions[i] = reflective_boundaries(positions[i], lb, ub)

        # Update best solution
        current_fitness = distance(positions[i], prey_pos[0], prey_pos[1])
        if current_fitness < best_fitness:
            best_fitness = current_fitness
            X_best = positions[i].copy()

    convergence_curve[iter] = best_fitness

    if iter % 10 == 0:
        print(f'Iteration: {iter}, Best Distance: {best_fitness:.4f}')

    plt.title(f'Aquila Optimizer - Iteration {iter+1}')
    plt.pause(0.01)

plt.close()

# Convergence plot
plt.figure(figsize=(10, 6))
plt.semilogy(convergence_curve, 'b', linewidth=2)
plt.xlabel('Iteration')
plt.ylabel('Best Distance to Prey')
plt.title('Modified Aquila Optimizer Convergence Curve')
plt.grid(True)
plt.show()

C:\Users\HARSHDIP\AppData\Local\Temp\ipykernel_24676\1957742337.py:101: DeprecationWarning: `np.math` is a deprecated alias for the standard library `math` module (Deprecated Numpy 1.25). Replace usages of `np.math` with `math`
  sigma = (np.math.gamma(1+beta)*np.sin(np.pi*beta/2)/(np.math.gamma((1+beta)/2)*beta*2**((beta-1)/2)))**(1/beta)


Iteration: 0, Best Distance: 4.2513
Iteration: 10, Best Distance: 2.7958
Iteration: 20, Best Distance: 2.7958
Iteration: 30, Best Distance: 2.7958
Iteration: 40, Best Distance: 2.7958
Iteration: 50, Best Distance: 2.7958
Iteration: 60, Best Distance: 2.7958
Iteration: 70, Best Distance: 2.7958
Iteration: 80, Best Distance: 2.7958
Iteration: 90, Best Distance: 2.7958
Iteration: 100, Best Distance: 2.7958
Iteration: 110, Best Distance: 2.7958
Iteration: 120, Best Distance: 2.7958
Iteration: 130, Best Distance: 2.7958
Iteration: 140, Best Distance: 2.7958
Iteration: 150, Best Distance: 2.7958
Iteration: 160, Best Distance: 2.7958
Iteration: 170, Best Distance: 2.7958
Iteration: 180, Best Distance: 2.7958
Iteration: 190, Best Distance: 2.7958
Iteration: 200, Best Distance: 2.7958
Iteration: 210, Best Distance: 2.7958
Iteration: 220, Best Distance: 2.7958
Iteration: 230, Best Distance: 2.7958
Iteration: 240, Best Distance: 2.7958
Iteration: 250, Best Distance: 2.7958
Iteration: 260, Best Di